In [1]:
import numpy as np
import pandas as pd
import torch
import os, sys, gc, time, warnings, pickle, random

from tsfmeta.data import MetaDataset,RawData, Md_utils,temporal_signal_split
from tsfmeta import utils
from tsfmeta import nn as MetaNN
from tsfmeta.experiments import meta_learning_run
from tqdm import tqdm

In [2]:

from math import ceil
from sklearn.preprocessing import LabelEncoder
dir_ = 'C:/M5/' # input only here
raw_data_dir = dir_+'M5-point/' 
processed_data_dir = 'E:/temp/'
TARGET = 'sales'         # Our main target
END_TRAIN = 1941-28         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [3]:
sale_df = pd.read_csv(raw_data_dir+'sales_train_validation.csv')
prices_df = pd.read_csv(raw_data_dir+'sell_prices.csv')
calendar_df = pd.read_csv(raw_data_dir+'calendar.csv')

In [7]:
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
sale_df = pd.melt(sale_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)
for col in index_columns:
    sale_df[col] = sale_df[col].astype('category')

icols = ['d', 'wm_yr_wk', 'wday', 'month',  'event_type_1', 'snap_CA',  'snap_TX', 'snap_WI',]

sale_df = sale_df.merge(calendar_df[icols], on=['d'], how='left')
sale_df = sale_df.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')
sale_df =  sale_df.drop(['wm_yr_wk','item_id'], axis=1)
sale_df['sell_price'] =  np.log(sale_df['sell_price'])
sale_df['d'] = sale_df['d'].apply(lambda x: x[2:]).astype(np.int16)  # Convert 'd' to int
sale_df.drop(sale_df.loc[sale_df['sell_price'].isna()].index, inplace=True)

le = LabelEncoder()
for col in ['event_type_1', 'snap_CA',  'snap_TX', 'snap_WI','dept_id','cat_id','store_id','state_id']:
    sale_df[col] = le.fit_transform(sale_df[col].values)
    

In [8]:
sale_df

,id,dept_id,cat_id,store_id,state_id,d,sales,wday,month,event_type_1,snap_CA,snap_TX,snap_WI,sell_price
7,HOBBIES_1_008_CA_1_validation,3,1,0,0,1,12,1,1,4,0,0,0,-0.776529
8,HOBBIES_1_009_CA_1_validation,3,1,0,0,1,2,1,1,4,0,0,0,0.444686
9,HOBBIES_1_010_CA_1_validation,3,1,0,0,1,0,1,1,4,0,0,0,1.153732
11,HOBBIES_1_012_CA_1_validation,3,1,0,0,1,0,1,1,4,0,0,0,1.788421
14,HOBBIES_1_015_CA_1_validation,3,1,0,0,1,4,1,1,4,0,0,0,-0.356675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,FOODS_3_823_WI_3_validation,2,0,9,2,1913,1,2,4,4,0,0,0,1.091923
58327366,FOODS_3_824_WI_3_validation,2,0,9,2,1913,0,2,4,4,0,0,0,0.908259
58327367,FOODS_3_825_WI_3_validation,2,0,9,2,1913,0,2,4,4,0,0,0,1.381282
58327368,FOODS_3_826_WI_3_validation,2,0,9,2,1913,3,2,4,4,0,0,0,0.246860


In [10]:
target = ['sales']
features_num = ['sell_price'] 
idx = ['id']  
tdx = ['d']
features_cat = ['event_type_1', 'snap_CA',  'snap_TX', 'snap_WI',
            'dept_id','cat_id','store_id','state_id', 'wday','month',] 
dynamic_known_features_num = ['sell_price']  ## should be part of features_num
dynamic_known_features_cat = []   ## should be part of features_cat

static_known_features = ['dept_id','cat_id','store_id','state_id']

df = Md_utils.Df_to_rawdata(df = sale_df,
                   idx = idx,
                   tdx = tdx,
                   target = target,
                   features_num = features_num,
                   features_cat= features_cat,
                   static_known_features = static_known_features,
                   dynamic_known_features_num = dynamic_known_features_num,
                   dynamic_known_features_cat = dynamic_known_features_cat,
                   )


In [13]:
#df = pickle.load(open(raw_data_dir +'M5_meta_rawdata' +'.pkl', 'wb'))

In [11]:
df_slots = df.data_segment(700,7,5)
df_slots[3].npdata.shape
df_slots[0].time_idx

Int64Index([1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216,
            ...
            1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913],
           dtype='int64', name='d', length=707)

In [13]:
for i in range(len(df_slots)):
    df_slots[i] = df_slots[i].Raw2Meta(H=7)
    #df_slots[i].reduce_mem_usage()

In [ ]:
# SES example
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from random import random

for slot in range(len(df_slots)):

    Nts,_, T = df_slots[slot].X.shape 
    _, H = df_slots[slot].Y.shape
    model = [SimpleExpSmoothing(df_slots[slot].X[i,0]).fit() for i in range(Nts)]
    model_pred = [model[i].predict(T,T + H -1) for i in range(Nts)]
    model_pred = np.stack(model_pred,0)
    df_slots[slot].add_Base_forecasts(model_pred,'SimpleExpSmoothing')

pickle.dump(df_slots, open(processed_data_dir +'M5_metadata_slots_book' +'.pkl', 'wb'), protocol= 4)

In [ ]:
# HEWS example
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from random import random

for slot in range(len(df_slots)):

    Nts,_, T = df_slots[slot].X.shape 
    _, H = df_slots[slot].Y.shape
    model = [ExponentialSmoothing(df_slots[slot].X[i,0], trend = 'add',damped_trend=True).fit() for i in range(Nts)]
    model_pred = [model[i].predict(T,T + H -1) for i in range(Nts)]
    model_pred = np.stack(model_pred,0)
    df_slots[slot].add_Base_forecasts(model_pred,'ExponentialSmoothing')
pickle.dump(df_slots, open(processed_data_dir +'M5_metadata_slots_book' +'.pkl', 'wb'), protocol= 4)

In [ ]:
#lgb direct example
import lightgbm as lgb
from sklearn.model_selection import train_test_split

for slot in range(len(df_slots)):
    print(slot)
    gdf = df_slots[slot].XZ_globalReg(L=7, rolling_step = 7, mode= 'Tree')

    H = gdf['Y_tr'].shape[-1]
    X_train, X_val, y_train, y_val, mask_train, mask_val,  =  train_test_split(np.concatenate([gdf['X_tr'],gdf['Z_tr']],-1)  ,gdf['Y_tr'] ,gdf['mask_tr'], test_size = 0.2, random_state =0)

    predicts = []
    for h in range(H): 
        dtrain = lgb.Dataset(X_train , label= y_train[:,h] ,weight = mask_train[:,h] )
        dval = lgb.Dataset(X_val , label= y_val[:,h] , weight = mask_val[:,h] )
        params = {
                'num_leaves': 1024,
                'objective': 'regression',
                'min_data_in_leaf': 50,
                'learning_rate': 0.02,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.7,
                'bagging_freq': 1,
                'metric': 'rmse',
                'num_threads': 8
            }
            
            
        MAX_ROUNDS = 2000
        bst = lgb.train(
                params, dtrain, num_boost_round=MAX_ROUNDS, 
                        valid_sets=[dtrain,dval], early_stopping_rounds=125, verbose_eval=30
                    )

        predicts.append( bst.predict(np.concatenate([gdf['X_ts'],gdf['Z_ts']],-1)  , num_iteration=bst.best_iteration or MAX_ROUNDS)[:,np.newaxis])

    predicts = np.concatenate(predicts,-1)
    df_slots[slot].add_Base_forecasts(predicts,'lightgbm_direct')
pickle.dump(df_slots, open(processed_data_dir +'M5_metadata_slots_book' +'.pkl', 'wb'), protocol= 4)

In [ ]:
#lgb mimo example
import lightgbm as lgb
from sklearn.model_selection import train_test_split

for slot in range(len(df_slots)):

    gdf = df_slots[slot].XZ_globalReg(L=14, rolling_step = 7, mode= 'Tree')
    H = gdf['Y_tr'].shape[-1]
    X = gdf['X_tr'].repeat(H,0)
    Y =  gdf['Y_tr'].reshape(-1)
    Z = np.stack([gdf['Z_tr'][:,(i*H):(i+1)*H].reshape(-1) for i in range(gdf['Z_tr'].shape[1]//H)],1)
    mask = gdf['mask_tr'].reshape(-1)

    
    X_train, X_val, y_train, y_val, mask_train, mask_val,  =  train_test_split(np.concatenate([X,Z],-1) ,Y ,mask, test_size = 0.2, random_state =0)

 
    X_test = gdf['X_ts'].repeat(H,0)
    Z_test = np.stack([gdf['Z_ts'][:,(i*H):(i+1)*H].reshape(-1) for i in range(gdf['Z_ts'].shape[1]//H)],1)
    mask_val = mask_val.reshape(-1)
    
    dtrain = lgb.Dataset(X_train , label= y_train ,weight = mask_train )
    dval = lgb.Dataset(X_val , label= y_val, weight = mask_val )
    
    params = {
            'num_leaves': 1024,
            'objective': 'regression',
            'min_data_in_leaf': 50,
            'learning_rate': 0.02,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.7,
            'bagging_freq': 1,
            'metric': 'rmse',
            'num_threads': 8
        }
        
        
    MAX_ROUNDS = 2000
    bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS, 
                    valid_sets=[dtrain,dval], early_stopping_rounds=125, verbose_eval=30
                )

    predicts = bst.predict( np.concatenate([X_test,Z_test],-1), num_iteration=bst.best_iteration or MAX_ROUNDS)

    predicts = predicts.reshape(len(gdf['X_ts']),H )
    df_slots[slot].add_Base_forecasts(predicts,'lightgbm_mimo')
pickle.dump(df_slots, open(processed_data_dir +'M5_metadata_slots_book' +'.pkl', 'wb'), protocol= 4)

In [7]:
df_slots[1].forecasters

['SimpleExpSmoothing',
 'ExponentialSmoothing',
 'lightgbm_direct',
 'lightgbm_mimo']

In [ ]:
# random forest example// too slow
from sklearn.ensemble import RandomForestRegressor

for slot in range(len(df_slots)):
    print(slot)
    gdf = df_slots[slot].XZ_globalReg(L=7, rolling_step = 7, mode= 'Tree')
    H = gdf['Y_tr'].shape[-1]
        
    regr = RandomForestRegressor(n_estimators=200 ,max_depth=10, random_state=0, n_jobs = 4 )
    regr.fit(X = np.nan_to_num(np.concatenate([gdf['X_tr'],gdf['Z_tr']],-1),nan = -1),
             y = np.nan_to_num(gdf['Y_tr'] ,nan = -1),
             sample_weight = gdf['mask_tr'][:,0])    
    predicts = regr.predict(X = np.nan_to_num(np.concatenate([gdf['X_ts'],gdf['Z_ts']],-1),nan = -1))   
    df_slots[slot].add_Base_forecasts(predicts,'rf_mimo')
pickle.dump(df_slots, open(processed_data_dir +'M5_metadata_slots_book' +'.pkl', 'wb'), protocol= 4)



In [3]:
### meta learning

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
df_slots = pickle.load( open(processed_data_dir +'M5_metadata_slots_book' +'.pkl', 'rb') )

In [5]:
learning_rate = 1e-5
batch_size = 256

meta_train = Md_utils.md_concat([df_slots[i] for i in range(1,5)],fidx = [0,1,2,3,4])
#meta_train.X = np.nan_to_num(meta_train.X, nan= -1)
meta_test = Md_utils.md_concat([df_slots[i] for i in range(1)],fidx = [0,1,2,3,4])

train_dataloader = meta_train.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
test_dataloader = meta_test.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

In [ ]:
preds = []
n_features = meta_train.Z.shape[1]
_, n_forecasters, horizon  = meta_train.B.shape
window_x = meta_train.X.shape[2]

n_known_features = len(meta_train.known_features)
Use_z = True

model = MetaNN.MetaComb(n_features= n_features, n_forecasters=n_forecasters,window_x=window_x,Use_z= Use_z,n_known_features= n_known_features, horizon=horizon).to(device)
loss_fn = torch.nn.MSELoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(),  lr=learning_rate)
epochs = 10
meta_learning = meta_learning_run(model,loss_fn, optimizer, train_dataloader,test_dataloader, device, epochs,True)
loss, pred = meta_learning.forecast(test_dataloader)
preds.append(np.expand_dims(pred.cpu().numpy(),1))

In [ ]:
## metaselection example
n_features = meta_train.Z.shape[1]
_, n_forecasters, horizon  = meta_train.B.shape
window_x = meta_train.X.shape[2]

model = MetaNN.MetaSelection(n_features= n_features, n_forecasters=n_forecasters,window_x=window_x, Use_z= Use_z,n_known_features=n_known_features,horizon=horizon).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(),  lr=learning_rate)
epochs = 20
meta_learning = meta_learning_run(model,loss_fn, optimizer, train_dataloader,test_dataloader, device, epochs)
loss, pred = meta_learning.forecast(test_dataloader)
preds.append(np.expand_dims(pred.cpu().numpy(),1))

In [ ]:
## metalossn example
n_features = meta_train.Z.shape[1]
_, n_forecasters, horizon  = meta_train.B.shape
window_x = meta_train.X.shape[2]

model = MetaNN.MetaLoss(n_features= n_features, n_forecasters=n_forecasters,window_x=window_x, Use_z= Use_z,n_known_features= n_known_features,horizon=horizon).to(device)
loss_fn = torch.nn.MSELoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(),  lr=learning_rate)
epochs = 30
meta_learning = meta_learning_run(model,loss_fn, optimizer, train_dataloader,test_dataloader, device, epochs)
loss, pred = meta_learning.forecast(test_dataloader)
preds.append(np.expand_dims(pred.cpu().numpy(),1))

In [8]:
pd.DataFrame(meta_test.evaluate(np.concatenate(preds,1),utils.metrics),index= meta_test.forecasters + ['meta_comb'])

,MAE,MSE,sMAPE1,MAPE,RMSSE
SimpleExpSmoothing,1.011806,4.680438,0.638231,0.675510,0.710221
ExponentialSmoothing,1.012420,4.685136,0.629418,0.676967,0.711432
lightgbm_direct,0.984526,3.708434,0.627456,0.710603,0.722578
lightgbm_mimo,0.984418,3.867940,0.629857,0.709617,0.711013
rf_mimo,1.024882,3.867784,0.623931,0.707739,0.757183
meta_comb,0.975732,3.743444,0.627249,0.708764,0.705908
